In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import json
import yaml

In [2]:
folder_to_harvest = Path("../data/douhet")

json_paths = folder_to_harvest.glob("**/*.json")

db = pd.DataFrame()

for data_path in json_paths:

    course_label =  data_path.stem.replace("_data", "")
    course, edition, ctx = course_label.split("_")

    associazione_path = Path(f"./data/{course}/{course}_{edition}/associazione.xlsx")
    associazione_df = pd.read_excel(associazione_path)
    
    with open(data_path, "r") as fin:
        
        data = json.loads(fin.read())
        root_key = list(data.keys())[0]
        
        sna_micro_stats_a = data[root_key]["sna"]["micro_stats_a"]
        sna_micro_stats_b = data[root_key]["sna"]["micro_stats_a"]
        sna_micro_sociogram = data[root_key]["sociogram"]["micro_stats"]
        
        sna_micro_stats_a_df = pd.DataFrame(sna_micro_stats_a).T.add_suffix("_a", axis=1)
        sna_macro_stats_b_df = pd.DataFrame(sna_micro_stats_b).T.add_suffix("_b", axis=1)
        sociogram_micro_stats_df = pd.DataFrame(sna_micro_sociogram).T
        
        course_df =  pd.concat([sna_micro_stats_a_df, sna_macro_stats_b_df, sociogram_micro_stats_df], axis=1)
        course_df = course_df.reset_index(names="id")
        course_df.insert(0, "corso", course)
        course_df.insert(1, "anno", edition)
        course_df.insert(3, "contesto", ctx)
        
        course_df = course_df.merge(associazione_df, left_on="id", right_on="lettera").drop("lettera", axis=1)
        columns_to_reorder =  list(course_df.columns)
        course_df = course_df.loc[:, [ *columns_to_reorder[:1], columns_to_reorder[-1], *columns_to_reorder[1:-1] ]]
    
    db = pd.concat([db, course_df])

db.to_excel("data_abgrid_per_analisi.xlsx", index=False)

In [3]:
db.pivot_table(index="corso", values=["affiliation_coeff_raw", "influence_coeff_raw"], aggfunc="mean")

,affiliation_coeff_raw,influence_coeff_raw
corso,,
ursa,2.62069,4.844828
vega,3.343284,4.902985
zenit,2.166667,4.416667
